# 第3回 (11/16) 宿題: CIFAR-10で高精度を目指そう
- 次回(11/30)までにいろいろ試して、高精度を目指してみて下さい！
- ベースラインとして、75%出るCNNを書いておきました。自由に書き換えてしまって構いません。
- 床爪のベストスコアは94.7%です。越えてみてください！

## CIFAR-10の読み込み

In [ ]:
import numpy as np
import chainer
from chainer import cuda, Variable, Chain, optimizers
import chainer.functions as F
import chainer.links as L
import cPickle as pickle

def unpickle(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

train = [unpickle('/data/ishimochi0/dataset/cifar-10-batches-py/data_batch_%d' %i) for i in range(1,6)]
X_train_ = np.concatenate([d['data'] for d in train]).reshape((-1, 3, 32, 32)).astype('float32') / 255.
y_train = np.concatenate([d['labels'] for d in train]).astype('int32')
N_train = len(X_train_)

test = unpickle('/data/ishimochi0/dataset/cifar-10-batches-py/test_batch')
X_test_ = test['data'].reshape((-1, 3, 32, 32)).astype('float32') / 255.
y_test = np.array(test['labels'], dtype='int32')
N_test = len(X_test_)

## データの前処理

In [ ]:
def preprocess(X_train_, X_test_):
    X_mean = np.mean(X_train_, axis=(0, 2, 3), keepdims=True)
    X_train = X_train_ - X_mean
    X_test = X_test_ - X_mean
    return X_train, X_test

## CNNクラス

In [ ]:
class CNN(Chain):
    def __init__(self):
        super(CNN, self).__init__(
            conv1 = L.Convolution2D(3, 32, 5, pad=2),
            conv2 = L.Convolution2D(32, 64, 5, pad=2),
            conv3 = L.Convolution2D(64, 128, 5, pad=2),
            fc4 = L.Linear(16*128, 128),
            fc5 = L.Linear(128, 10)
        )
    
    def __call__(self, x, t):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(h, 2)
    
        h = F.relu(self.fc4(h))
        self.y = self.fc5(h)
        self.loss = F.softmax_cross_entropy(self.y, t)
        
        return self.loss

## モデルの定義とoptimizerの設定

In [ ]:
model = CNN() 
model.to_gpu()
optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))

## CIFAR-10の学習

In [ ]:
X_train, X_test = preprocess(X_train_, X_test_)

n_epoch = 20
batchsize = 100

for epoch in range(n_epoch):
    print 'epoch %d |' % epoch,
    
    # Training
    sum_loss = 0
    pred_y = []
    perm = np.random.permutation(N_train)
    
    for i in xrange(0, N_train, batchsize):
        x = Variable(cuda.to_gpu(X_train[perm[i: i+batchsize]]))
        t = Variable(cuda.to_gpu(y_train[perm[i: i+batchsize]]))
        
        optimizer.update(model, x, t)
        sum_loss += cuda.to_cpu(model.loss.data) * len(x.data)
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1)) 
    
    loss = sum_loss / N_train
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_train[perm]]) / N_train
    print 'Train loss %.3f, accuracy %.4f |' %(loss, accuracy), 
    
    
    # Testing
    sum_loss = 0
    pred_y = []
    
    for i in xrange(0, N_test, batchsize):
        x = Variable(cuda.to_gpu(X_test[i: i+batchsize]))
        t = Variable(cuda.to_gpu(y_test[i: i+batchsize]))
        
        sum_loss += cuda.to_cpu(model(x, t).data) * len(x.data)
        pred_y.extend(np.argmax(cuda.to_cpu(model.y.data), axis=1))

    loss = sum_loss / N_test
    accuracy = np.sum(np.eye(10)[pred_y] * np.eye(10)[y_test]) / N_test
    print 'Test loss %.3f, accuracy %.4f' %(loss, accuracy) 